In [4]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models


# This code was tested with TensorFlow v1.8
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 2.1.0


In [5]:
url = r"C:\Users\gonza\OneDrive\Documentos\datascience-thebridge-master\ejercicios gonzalo\DATA\bbc-text.csv"
data = pd.read_csv(url)
data.head(4)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...


In [7]:
data["category"].value_counts(normalize=True)

sport            0.229663
business         0.229213
politics         0.187416
tech             0.180225
entertainment    0.173483
Name: category, dtype: float64

In [8]:
train_size = int(len(data) * .8)
print(f"Train size: {train_size}")
print(f"Test size: {len(data)-train_size}")

Train size: 1780
Test size: 445


In [9]:
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

### Data preparation
There's some work to be done in order for our data to be ready for training.

#First we'll split the data into training and test sets.
#Then we'll tokenize the words (text), and then convert them to a numbered index.
#Next we'll do the same for the labels (categories), by using the LabelEncoder utility.
#Finally, we'll convert the labels to a one-hot representation.

In [10]:
train_cat, test_cat = train_test_split(data["category"], train_size)
train_text, test_text = train_test_split(data["text"],train_size)

In [11]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, char_level=False)

In [12]:
tokenize.fit_on_texts(train_text) # fit tokenizer to our trian data
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [ ]:
# use sklearn to convert label strings to numbered index (ojo solo usamos transform en test no fit)
